In [1]:
"""
Created on Mon Feb  7 14:10:42 2022

@author: Okhrimchuk Roman
for Sierentz Global Merchants

Test task
"""

# TODO Import the necessary libraries
import pandas as pd
import numpy as np

# TODO Import the dataset 

path = r'./data/weather_dataset.data'
data = pd.read_csv(path, sep=';')
data.head()

,Yr,Mo,Dy,loc1,loc2,loc3,loc4,loc5,loc6,loc7,loc8,loc9,loc10,loc11,loc12
0,61,1,1,15.04,14.96,13.17,9.29,NaN,9.87,13.67,10.25,10.83,12.58,18.50,15.04
1,61,1,2,14.71,NaN,10.83,6.50,12.62,7.67,11.50,10.04,9.79,9.67,17.54,13.83
2,61,1,3,18.50,16.88,12.33,10.13,11.17,6.17,11.25,NaN,8.50,7.67,12.75,12.71
3,61,1,4,10.58,6.63,11.75,4.58,4.54,2.88,8.63,1.79,5.83,5.88,5.46,10.88
4,61,1,5,13.33,13.25,11.42,6.17,10.71,8.21,11.92,6.54,10.92,10.34,12.92,11.83


In [2]:
# TODO  Assign it to a variable called data and replace the first 3 columns by a proper datetime index

# TODO Check if everything is okay with the data. Create functions to delete/fix rows with strange cases and apply them

# TODO Write a function in order to fix date (this relate only to the year info) and apply it
dates = '19' + data.Yr.astype(str) + '-' + data.Mo.astype(str) + '-' + data.Dy.astype(str)
data['date'] = dates
data.head()

,Yr,Mo,Dy,loc1,loc2,loc3,loc4,loc5,loc6,loc7,loc8,loc9,loc10,loc11,loc12,date
0,61,1,1,15.04,14.96,13.17,9.29,NaN,9.87,13.67,10.25,10.83,12.58,18.50,15.04,1961-1-1
1,61,1,2,14.71,NaN,10.83,6.50,12.62,7.67,11.50,10.04,9.79,9.67,17.54,13.83,1961-1-2
2,61,1,3,18.50,16.88,12.33,10.13,11.17,6.17,11.25,NaN,8.50,7.67,12.75,12.71,1961-1-3
3,61,1,4,10.58,6.63,11.75,4.58,4.54,2.88,8.63,1.79,5.83,5.88,5.46,10.88,1961-1-4
4,61,1,5,13.33,13.25,11.42,6.17,10.71,8.21,11.92,6.54,10.92,10.34,12.92,11.83,1961-1-5


In [3]:
# TODO Set the right dates as the index. Pay attention at the data type, it should be datetime64[ns]

data['date'] = pd.to_datetime(dates)
data.drop(columns=['Yr', 'Mo', 'Dy'], inplace=True)

print(data.dtypes)

loc1             object
loc2             object
loc3             object
loc4             object
loc5             object
loc6             object
loc7             object
loc8             object
loc9             object
loc10            object
loc11            object
loc12            object
date     datetime64[ns]
dtype: object


In [4]:
# TODO Compute how many values are missing for each location over the entire record
locations = [col for col in data.columns if col.startswith('loc')]
for loc in locations:
    nans = data[loc].isna().sum()
    print(f'There are {nans} missing values in the column {loc}')

There are 6 missing values in the column loc1
There are 3 missing values in the column loc2
There are 3 missing values in the column loc3
There are 5 missing values in the column loc4
There are 2 missing values in the column loc5
There are 0 missing values in the column loc6
There are 3 missing values in the column loc7
There are 2 missing values in the column loc8
There are 4 missing values in the column loc9
There are 1 missing values in the column loc10
There are 0 missing values in the column loc11
There are 4 missing values in the column loc12


In [5]:
# TODO Compute how many non-missing values there are in total
all_values = sum(data.count())
print(f'There are {all_values} non-missing values in total')

There are 85429 non-missing values in total


In [6]:
# TODO Calculate the mean windspeeds of the windspeeds over all the locations and all the times

#replacing erroneous entries with NaNs and then filling them with column means
bad_strings = ['None', 'NONE', '-123*None', 'nodata', '1.0k']

for loc in locations:
    data[loc] = data[loc].apply(lambda x: str(x).replace(',', '.'))
    data[loc] = data[loc].replace(bad_strings, np.nan).astype(float)
    data[loc] = data[loc].fillna(data[loc].mean())

#replacing all negative numbers with zero
for loc in locations:
    data[loc].clip(lower=0, inplace=True)
    
#replacing outliers in location 9 with median
data.loc9 = np.where(data.loc9>25.88, data.loc9.median(), data.loc9)

data[locations].mean()

loc1     12.362987
loc2     10.644314
loc3     11.657946
loc4      6.304478
loc5     10.457101
loc6      7.092254
loc7      9.793966
loc8      8.491293
loc9      8.490678
loc10     8.707115
loc11    13.117172
loc12    15.598111
dtype: float64

In [17]:
# TODO Create a DataFrame called loc_stats and calculate the min, max and mean windspeeds and standard deviations of the windspeeds at each location over all the days

pd.set_option('display.float_format', lambda x: '%.2f' % x)

loc_stats = pd.DataFrame({
    'min': data[locations].min(),
    'max': data[locations].max(),
    'mean': data[locations].mean(),
    'std': data[locations].std(),
})

loc_stats

,min,max,mean,std
loc1,0.67,35.80,12.36,5.62
loc2,0.21,33.37,10.64,5.27
loc3,0.00,33.84,11.66,5.01
loc4,0.00,28.46,6.30,3.60
loc5,0.13,37.54,10.46,4.93
loc6,0.00,26.16,7.09,3.97
loc7,0.00,30.37,9.79,4.98
loc8,0.00,31.08,8.49,4.50
loc9,0.00,25.88,8.49,4.17
loc10,0.04,28.21,8.71,4.50


In [8]:
# TODO Find the average windspeed in January for each location

data[data.date.dt.month == 1].mean()

C:\Users\oleks\AppData\Local\Temp/ipykernel_17076/3927772603.py:3: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  data[data.date.dt.month == 1].mean()


loc1    14.84
loc2    12.91
loc3    13.30
loc4     7.20
loc5    11.66
loc6     8.05
loc7    11.82
loc8     9.51
loc9     9.54
loc10   10.05
loc11   14.55
loc12   18.03
dtype: float64

In [14]:
# TODO Downsample the record to a yearly frequency for each location

annual_data = data.set_index('date').resample('Y').mean()
annual_data.head()

,loc1,loc2,loc3,loc4,loc5,loc6,loc7,loc8,loc9,loc10,loc11,loc12
date,,,,,,,,,,,,
1961-12-31,12.30,10.35,11.32,6.92,10.88,7.73,9.73,8.86,8.64,9.83,13.50,13.70
1962-12-31,12.25,10.11,11.73,6.96,10.66,7.39,11.02,8.79,8.32,9.68,12.93,14.33
1963-12-31,12.81,10.84,12.54,7.33,11.72,8.43,11.08,10.34,8.90,10.22,13.64,15.00
1964-12-31,12.36,10.92,12.10,6.79,11.45,7.57,10.26,9.47,7.79,10.21,13.74,14.91
1965-12-31,12.45,11.08,11.85,6.86,11.02,7.48,10.62,8.88,7.91,9.92,12.96,15.59


In [15]:
# TODO Downsample the record to a monthly frequency for each location

monthly_data = data.set_index('date').resample('M').mean()
monthly_data.head()

,loc1,loc2,loc3,loc4,loc5,loc6,loc7,loc8,loc9,loc10,loc11,loc12
date,,,,,,,,,,,,
1961-01-31,14.76,11.94,13.43,7.74,11.03,8.59,11.18,9.22,9.09,10.11,13.88,14.70
1961-02-28,16.27,14.98,14.07,9.13,13.85,10.94,11.89,11.85,11.82,12.71,18.58,15.41
1961-03-31,10.94,11.30,10.75,7.25,10.51,8.87,9.64,9.83,10.16,11.25,16.41,15.72
1961-04-30,10.72,9.43,10.00,5.83,8.44,6.50,6.93,7.09,7.34,7.24,11.15,10.28
1961-05-31,9.86,8.91,10.82,5.92,9.49,6.57,7.67,8.18,8.04,8.50,11.90,12.01


In [16]:
# TODO Downsample the record to a weekly frequency for each location

weekly_data = data.set_index('date').resample('W').mean()
weekly_data.head()

,loc1,loc2,loc3,loc4,loc5,loc6,loc7,loc8,loc9,loc10,loc11,loc12
date,,,,,,,,,,,,
1961-01-01,15.04,14.96,13.17,9.29,10.46,9.87,13.67,10.25,10.83,12.58,18.50,15.04
1961-01-08,13.54,11.37,10.49,6.42,9.47,6.44,11.06,6.88,8.43,8.50,12.48,13.24
1961-01-15,12.47,8.97,11.96,4.63,7.35,5.07,7.54,6.82,5.71,7.57,11.13,11.02
1961-01-22,13.20,9.86,12.98,6.33,9.18,7.42,9.26,7.88,7.15,8.12,9.82,11.43
1961-01-29,18.81,16.14,18.23,12.72,17.43,14.83,15.53,15.16,14.48,15.64,20.93,22.53


In [21]:
# TODO Calculate the min, max and mean windspeeds and standard deviations of the windspeeds across all locations for each week (assume that the first week starts on January 2 1961) for the first 21 weeks

agg_weekly_data = data.set_index('date').resample('W').agg(['min', 'max', 'mean', 'std']).head(21)
agg_weekly_data

loc1                   loc2                   loc3        ...  \
             min   max  mean  std   min   max  mean  std   min   max  ...   
date                                                                  ...   
1961-01-01 15.04 15.04 15.04  NaN 14.96 14.96 14.96  NaN 13.17 13.17  ...   
1961-01-08 10.58 18.50 13.54 2.63  6.63 16.88 11.37 3.62  7.62 12.33  ...   
1961-01-15  9.04 19.75 12.47 3.56  3.54 12.08  8.97 3.15  7.08 19.50  ...   
1961-01-22  4.92 19.83 13.20 5.34  3.42 14.37  9.86 3.84  7.29 20.79  ...   
1961-01-29 12.36 25.04 18.81 5.08  9.96 23.91 16.14 5.17 12.67 25.84  ...   
1961-02-05 10.58 24.21 16.83 5.25  9.46 24.21 15.46 5.19  9.04 19.70  ...   
1961-02-12 16.00 24.54 19.68 3.59 11.54 21.42 16.42 3.61 13.67 21.34  ...   
1961-02-19  6.04 22.50 15.13 5.06 11.63 20.17 15.09 3.58  6.13 17.25  ...   
1961-02-26  7.79 25.80 15.22 7.02  7.08 21.50 13.63 5.15  6.08 22.42  ...   
1961-03-05 10.96 13.33 12.10 1.00  8.83 17.00 12.95 2.85  8.17 13.67  ...   
1961-03-12  4.88 14.79  9.80 3.59  8.08 16.96 11.58 3.23  7.54 16.38  ...   
1961-03-19  4.92 16.88 11.91 3.86  9.46 15.54 13.50 2.35  5.25 13.96  ...   
1961-03-26  6.29 15.00  9.57 3.61  2.58 11.63  8.39 3.66  4.79 15.63  ...   
1961-04-02  5.88 18.25 10.76 5.05  3.50 16.29  8.85 4.69  5.09 14.96  ...   
1961-04-09  4.50 18.12 11.96 4.60  7.04 14.62 10.65 2.85  9.29 18.29  ...   
1961-04-16  4.71 15.50  8.97 3.94  4.83 12.25  8.00 2.61  3.92 15.79  ...   
1961-04-23  4.00 21.09 12.62 5.68  3.71 15.41 10.44 4.63  3.33 17.00  ...   
1961-04-30  4.08 16.29 10.12 4.35  6.50 14.46  9.80 2.87  2.54 14.96  ...   
1961-05-07  9.87 23.00 15.37 5.03 10.29 19.79 13.97 3.75  8.42 21.21  ...   
1961-05-14  3.54 12.79  7.77 3.37  3.96 15.12  8.71 3.78  4.63 12.33  ...   
1961-05-21  4.88 15.04  8.23 3.63  3.58 10.64  6.35 2.94  5.91 15.96  ...   

           loc10      loc11                  loc12                   
            mean  std   min   max  mean  std   min   max  mean  std  
date                                                                 
1961-01-01 12.58  NaN 18.50 18.50 18.50  NaN 15.04 15.04 15.04  NaN  
1961-01-08  8.50 1.70  5.46 17.54 12.48 4.35 10.88 16.46 13.24 1.77  
1961-01-15  7.57 4.08  5.25 20.71 11.13 5.55  5.17 16.92 11.02 4.69  
1961-01-22  8.12 4.78  6.50 15.92  9.82 3.63  6.79 17.96 11.43 4.24  
1961-01-29 15.64 3.71 14.04 27.71 20.93 5.21 17.50 27.63 22.53 3.87  
1961-02-05  9.46 2.84  9.17 19.33 14.01 4.21  7.17 19.25 11.94 4.34  
1961-02-12 14.44 1.75 15.21 26.38 21.83 4.06 17.04 21.84 19.16 1.83  
1961-02-19 13.54 2.53 14.09 29.63 21.17 5.91 10.96 22.58 16.58 4.69  
1961-02-26 12.73 4.92  9.59 23.21 16.30 5.09  6.67 23.87 14.32 6.18  
1961-03-05 12.37 1.59 11.58 23.45 17.84 4.33  8.83 17.54 14.19 2.83  
1961-03-12 10.46 3.66 10.21 22.71 16.70 4.36  5.54 22.54 14.42 5.77  
1961-03-19 11.63 3.10 11.29 22.79 19.35 3.78 11.34 22.95 16.23 4.33  
1961-03-26 11.48 2.54  8.25 21.34 14.04 4.32 13.13 22.50 18.13 3.70  
1961-04-02  9.63 3.19  7.21 18.63 13.47 4.18  7.17 19.58 13.90 3.92  
1961-04-09  7.24 2.34  7.62 17.16 11.71 3.15  7.21 15.34 11.37 2.60  
1961-04-16  6.18 2.16  5.75 16.17  9.48 3.64  5.66 12.87  8.69 2.75  
1961-04-23  9.55 3.35  6.75 19.21 13.62 4.74  4.96 20.46 12.47 5.91  
1961-04-30  6.12 2.84  5.13 13.04  9.72 2.95  2.67 17.50  8.64 5.11  
1961-05-07 11.59 3.62  4.79 28.08 17.55 8.00  3.83 26.58 14.57 7.73  
1961-05-14  7.82 5.46  6.54 18.66 10.42 3.97  3.33 26.30 10.38 7.86  
1961-05-21  7.11 2.22  6.63 12.00  9.62 1.98  5.91 14.96 10.61 3.31  

[21 rows x 48 columns]